In [24]:
import os
import sys
import math
import random
import joblib
import datetime
import numpy as np
import pandas as pd
from tqdm import tqdm
import warnings
warnings.filterwarnings(action='ignore')

In [2]:
sale = pd.read_excel(os.path.join('..', '..', '0.Data', '01_제공데이터', 'sale_data_v05_0828.xlsx'))

In [16]:
sale['방송날'] = sale['방송일시'].dt.date
sale['방송월'] = sale['방송일시'].dt.month
sale['방송일'] = sale['방송일시'].dt.day
sale['방송시간'] = sale['방송일시'].dt.hour

In [17]:
sale_a = sale.loc[sale['방송일시'] < '2019-11-01 02:00:00']

In [18]:
sale_b = sale.loc[sale['방송일시'] > '2019-11-01 02:00:00']

In [56]:
sale_a['방송날'] = sale_a.apply(lambda x : datetime.datetime.combine(x['방송날'], datetime.time(x['방송시간'])), axis = 1)

In [129]:
temp_a = sale_a.drop_duplicates(['방송월', '방송일', '방송시간'])[['방송날', '마더코드', '상품명', 'NEW상품코드', '상품군']]

In [28]:
sale_b['new_방송일시'] = sale_b['방송일시'] - datetime.timedelta(minutes = 20)

In [42]:
sale_b['방송날'] = sale_b['new_방송일시'].dt.date
sale_b['방송월'] = sale_b['new_방송일시'].dt.month
sale_b['방송일'] = sale_b['new_방송일시'].dt.day
sale_b['방송시간'] = sale_b['new_방송일시'].dt.hour

In [49]:
d = datetime.date(2018, 7, 28)
t = datetime.time(18)
dt = datetime.datetime.combine(d, t)
print(dt)

2018-07-28 18:00:00


In [52]:
sale_b['방송날'] = sale_b.apply(lambda x : datetime.datetime.combine(x['방송날'], datetime.time(x['방송시간'])), axis = 1)

In [130]:
temp_b = sale_b.drop_duplicates(['방송월', '방송일', '방송시간'])[['방송날', '마더코드', '상품명', 'NEW상품코드', '상품군']]

In [131]:
df = pd.concat([temp_a, temp_b], axis = 0)

In [132]:
df

,방송날,마더코드,상품명,NEW상품코드,상품군
0,2019-01-01 06:00:00,100346,테이트 남성 셀린니트3종,865.0,의류
6,2019-01-01 07:00:00,100305,오모떼 레이스 파운데이션 브라,632.0,속옷
9,2019-01-01 08:00:00,100808,CERINI by PAT 남성 소프트 기모 릴렉스팬츠,1079.0,의류
12,2019-01-01 09:00:00,100816,보코 리버시블 무스탕,378.0,의류
15,2019-01-01 10:00:00,100809,CERINI by PAT 남성 풀패키지 기모니트 3종,1089.0,의류
...,...,...,...,...,...
38283,2019-12-31 21:00:00,100155,일시불 LG전자 매직스페이스 냉장고,1123.0,가전
38289,2019-12-31 22:00:00,100543,넥센타이어 렌탈서비스,NaN,무형
38292,2019-12-31 23:00:00,100448,무이자쿠첸압력밥솥 10인용,818.0,주방
38304,2020-01-01 00:00:00,100073,삼성화재 행복한파트너 주택화재보험(1912),NaN,무형


In [133]:
df['방송일'] = pd.to_datetime(df['방송날']).dt.date
df['방송시간'] = pd.to_datetime(df['방송날']).dt.hour

In [134]:
df = df.sort_values(['방송시간', '방송날'])

In [152]:
pd.pivot_table(df, columns = '방송날', index = '상품군', values = 'NEW상품코드')

방송날,2019-01-01 06:00:00,2019-01-01 07:00:00,2019-01-01 08:00:00,2019-01-01 09:00:00,2019-01-01 10:00:00,2019-01-01 11:00:00,2019-01-01 12:00:00,2019-01-01 13:00:00,2019-01-01 14:00:00,2019-01-01 15:00:00,...,2019-12-31 12:00:00,2019-12-31 14:00:00,2019-12-31 15:00:00,2019-12-31 16:00:00,2019-12-31 17:00:00,2019-12-31 18:00:00,2019-12-31 19:00:00,2019-12-31 20:00:00,2019-12-31 21:00:00,2019-12-31 23:00:00
상품군,,,,,,,,,,,,,,,,,,,,,
가구,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,132.0,NaN,NaN,NaN
가전,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1123.0,NaN
건강기능,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1254.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
농수축,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,407.0,...,NaN,NaN,1327.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
생활용품,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,49.0,NaN,NaN
속옷,NaN,632.0,NaN,NaN,NaN,831.0,NaN,629.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
의류,865.0,NaN,1079.0,378.0,1089.0,NaN,NaN,NaN,23.0,NaN,...,NaN,NaN,NaN,NaN,NaN,250.0,NaN,NaN,NaN,NaN
이미용,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
잡화,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [151]:
data = df.sort_values(['상품군', '방송일'])[['상품군', 'NEW상품코드']].reset_index(drop = True)
data

,상품군,NEW상품코드
0,가구,341.0
1,가구,341.0
2,가구,341.0
3,가구,341.0
4,가구,449.0
...,...,...
7289,침구,370.0
7290,침구,370.0
7291,침구,369.0
7292,침구,368.0


In [153]:
data['상품군'].value_counts()

농수축     1228
주방      1080
의류      1061
속옷       976
잡화       598
가전       573
생활용품     494
이미용      425
무형       370
건강기능     222
가구       198
침구        69
Name: 상품군, dtype: int64

In [224]:
data = df[['방송일', '방송시간', 'NEW상품코드']].reset_index(drop = True)

In [225]:
pd.pivot_table(data, index = '방송시간', columns = '방송일', values = 'NEW상품코드')

방송일,2019-01-01,2019-01-02,2019-01-03,2019-01-04,2019-01-05,2019-01-06,2019-01-07,2019-01-08,2019-01-09,2019-01-10,...,2019-12-22,2019-12-23,2019-12-24,2019-12-25,2019-12-26,2019-12-27,2019-12-28,2019-12-29,2019-12-30,2019-12-31
방송시간,,,,,,,,,,,,,,,,,,,,,
0,NaN,30.0,865.0,256.0,489.0,402.0,476.0,1099.0,530.0,402.0,...,NaN,1290.0,157.0,1039.0,NaN,1223.0,NaN,NaN,65.0,652.0
1,NaN,140.0,928.0,322.0,4.0,1348.0,84.0,928.0,4.0,140.0,...,974.0,781.0,651.0,1074.0,1276.0,1258.0,1290.0,543.0,780.0,174.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,865.0,322.0,597.0,1328.0,481.0,742.0,139.0,481.0,378.0,269.0,...,304.0,543.0,250.0,79.0,609.0,842.0,687.0,292.0,637.0,838.0
7,632.0,741.0,747.0,780.0,535.0,255.0,607.0,241.0,741.0,241.0,...,1078.0,NaN,930.0,244.0,1362.0,1323.0,79.0,328.0,1323.0,1074.0
8,1079.0,632.0,607.0,167.0,241.0,162.0,378.0,812.0,139.0,530.0,...,847.0,648.0,847.0,823.0,47.0,587.0,471.0,1162.0,1362.0,1162.0
9,378.0,1154.0,154.0,236.0,644.0,378.0,402.0,607.0,235.0,629.0,...,773.0,47.0,526.0,622.0,1046.0,38.0,622.0,1202.0,526.0,69.0
10,1089.0,819.0,629.0,824.0,378.0,632.0,629.0,824.0,607.0,235.0,...,982.0,1314.0,746.0,315.0,1287.0,847.0,16.0,332.0,1287.0,1039.0


In [226]:
data['방송시간'].value_counts()

21    365
9     365
15    365
11    365
8     365
22    365
16    365
10    365
20    365
1     365
19    365
17    365
13    365
12    364
23    364
6     364
18    364
7     364
0     364
14    363
5       1
2       1
Name: 방송시간, dtype: int64

In [227]:
data = data.loc[data['NEW상품코드'].notnull()]

In [228]:
data['NEW상품코드'] = data['NEW상품코드'].astype(int)

In [229]:
data = data[(data['방송시간'] != 5) & (data['방송시간'] != 2)]

In [230]:
data[['방송시간', 'NEW상품코드']].to_csv('./nsshop_ver2.csv', encoding = 'cp949')

In [231]:
data

,방송일,방송시간,NEW상품코드
0,2019-01-02,0,30
1,2019-01-03,0,865
2,2019-01-04,0,256
3,2019-01-05,0,489
4,2019-01-06,0,402
...,...,...,...
7285,2019-12-23,23,726
7288,2019-12-26,23,979
7291,2019-12-29,23,458
7292,2019-12-30,23,920


In [213]:
data['방송월'] = pd.to_datetime(data['방송일']).dt.month

In [214]:
data['code'] = data['방송월'].astype(str) + '-' + data['방송시간'].astype(str)

In [202]:
meta = data[['code', '방송월']].drop_duplicates().reset_index(drop = True).reset_index()

In [219]:
res = data.merge(meta, on = 'code', how = 'left')[['index', 'NEW상품코드']]

In [221]:
res.to_csv('./nsshop.csv', encoding = 'cp949')

In [222]:
res

,index,NEW상품코드
0,0,30
1,0,865
2,0,256
3,0,489
4,0,402
...,...,...
6917,239,726
6918,239,979
6919,239,458
6920,239,920
